In [ ]:
!pip3 install -q numpy==1.22.4
!pip3 install -q pandas==2.1.0
!pip3 install -q matplotlib==3.7.2
!pip3 install -q seaborn==0.12.2
!pip3 install phik

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import phik

import sweetviz as sv

import os
import sys
import warnings


In [ ]:
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
np.set_printoptions(threshold=sys.maxsize)

warnings.filterwarnings(action='ignore')

In [ ]:
palette = sns.color_palette('Set2')
sns.palplot(palette)

# Загрузка датасета и первичный осмотр данных

In [ ]:
path1 = '/datasets/train.csv'
path2 = '/datasets/test.csv'

def read_file(path):
    df = pd.DataFrame()
    if os.path.exists(path):
        df = pd.read_csv(path, sep=',')
    elif os.path.exists(path[1:]):
        df = pd.read_csv(path[1:], sep=',')
    else:
        print('No such file or directory') 
        raise FileNotFoundError('No such file or directory')
    return df

df_train = read_file(path1)
df_test = read_file(path2)

In [ ]:
df_train.head(10)

In [ ]:
df_test.head(10)

In [ ]:
display(df_train.describe(), df_test.describe())

In [ ]:
display(df_train.describe(include=['O']), df_test.describe(include=['O']))

---

# Анализ имеющихся проблем с данными

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
df_train.isna().sum()

In [ ]:
df_test.isna().sum()

In [ ]:
def pass_value_barh(dfg, set_name, color='#1f77b4'):
    try:
        ax = (
            (dfg.isna().mean()*100)
            .to_frame()
            .rename(columns = {0:'space'})
            .query('space > 0')
            .sort_values(by = 'space', ascending = True)
            .plot(kind = 'barh', figsize = (19,6), rot = 0, legend = False, fontsize = 12, color=color)
        )
        ax.set_title(f'Percentage of Missing Values in {set_name} Columns\n', fontsize=20, color='steelblue')
        ax.set_xlabel('Percentage Missing', fontsize = 16)
        ax.set_ylabel('Columns', fontsize = 16)
        ax.grid(axis='x', linestyle='--', alpha=0.5)
        ax.bar_label(ax.containers[0], label_type='edge', fmt='%.2f%%')
    except:
        print('пропусков не осталось :) или произошла ошибка в первой части функции ')

In [ ]:
pass_value_barh(df_train, 'Train Set')

In [ ]:
pass_value_barh(df_test, 'Test Set', color='#2ca02c')

In [ ]:
print(df_train.duplicated().sum(), df_test.duplicated().sum())

---

# Предобработка тренировочного датасета и первичный анализ

In [ ]:
def plot_size(column, labels, explode, palette):
    values = df_train[column].value_counts()
    
    lb = ''
    if labels == '':
        lb = values.index
    else:
        lb = labels
    
    fig, ax = plt.subplots(2, 1, figsize=(12, 12), tight_layout=True)

    ax[0].bar(lb, values, color=palette)
    ax[0].grid(True, color='grey', axis='y', linestyle='-.', linewidth=0.5, alpha=0.6)
    ax[0].set_xlabel('Home Planet', fontsize=16)
    ax[0].set_ylabel('Number of transported clients', fontsize=14)
    ax[0].set_title(f'Total number of transported clients over {column}', fontsize=14)
    ax[0].bar_label(ax[0].containers[0], \
                 label_type='center', fmt='%.2f', fontsize=14, color='white')
    ax[0].tick_params(axis='x', labelsize=11)
    ax[0].tick_params(axis='y', labelsize=11)
    
    ax[1].pie(values, labels=lb, autopct='%1.2f%%', explode=explode, textprops={'fontsize':14}, startangle=100, colors=palette)
    ax[1].set_title(f'Percentage of {column} by the number of flights', fontsize=16)

    plt.show()
    
    return

In [ ]:
def plot_size_multiple(dfs, column, labels, explode, palette, width=15, height=6):
    df_number = len(dfs)
    fig, axs = plt.subplots(df_number, 2, figsize=(width, height * df_number), tight_layout=True)

    for i, df in enumerate(dfs):
        values = df[column].value_counts()
        
        lb = ''
        if labels == '':
            lb = values.index
        else:
            lb = labels

        axs[0][i].bar(lb, values, color=palette)
        axs[0][i].grid(True, color='grey', axis='y', linestyle='-.', linewidth=0.5, alpha=0.6)
        axs[0][i].set_xlabel('Home Planet', fontsize=14)
        axs[0][i].set_ylabel('Number of transported clients', fontsize=12)
        axs[0][i].set_title(f'Total number of transported clients over {column}', fontsize=12)
        axs[0][i].bar_label(axs[0][i].containers[0], \
                    label_type='center', fmt='%.2f', fontsize=12, color='white')
        axs[0][i].tick_params(axis='x', labelsize=11)
        axs[0][i].tick_params(axis='y', labelsize=11)
        
        axs[1][i].pie(values, labels=lb, autopct='%1.2f%%', explode=explode, textprops={'fontsize':12}, startangle=100, colors=palette)
        axs[1][i].set_title(f'Percentage of {column} by the number of flights', fontsize=16)

    plt.show()
    
    return

In [ ]:
def plot_tab(column, horizontal=False):
    fig, ax = plt.subplots(figsize=(6, 5))
    tab = pd.crosstab(df_train[column], df_train['Transported'])
    display(tab)
    
    if horizontal:
        tab.div(tab.sum(axis=1), axis=0).plot(kind="barh", stacked=True, color=[palette[1], palette[2]], ax=ax)
        ax.set_xlabel('Proportion', fontsize=12)
        ax.set_ylabel(column, fontsize=12)
        ax.grid(True, color='grey', axis='x', linestyle='-.', linewidth=0.5, alpha=0.6)
        ax.set_xlim(0, 1)
        ax.set_xticks([0.0, 0.2, 0.4, 0.5, 0.6, 0.8, 1.0])
        ax.legend(title='Transported', loc='upper left', labels=['False', 'True'], bbox_to_anchor=(1, 1))
        ax.axvline(x=0.5, color='green', linestyle='--', alpha=0.7)
    else:
        tab.div(tab.sum(axis=1), axis=0).plot(kind="bar", stacked=True, color=[palette[1], palette[2]], ax=ax)
        ax.set_xlabel(column)
        ax.set_ylabel('Proportion')
        ax.grid(True, color='grey', axis='y', linestyle='-.', linewidth=0.5, alpha=0.6)
        ax.legend(title='Transported', loc='upper left', labels=['False', 'True'], bbox_to_anchor=(1, 1))
        
    plt.xticks(rotation=0)
    ax.bar_label(ax.containers[0], label_type='center', fmt='%.2f')
    ax.bar_label(ax.containers[1], label_type='center', fmt='%.2f')
    ax.set_title(f'Stacked Bar Chart of {column} vs. Transported')
    
    plt.show()
    
    return

---

## Transported

In [ ]:
df_train['Transported'].value_counts(dropna=False)

In [ ]:
df_train['Transported'].dtype

In [ ]:
passanger_transported = df_train['Transported'].value_counts()

plt.figure(figsize=(8, 6))

plt.pie(passanger_transported, labels=['Yes', 'No'],
                                    autopct='%1.2f%%', explode=(0.01, 0), textprops={'fontsize':14},
                                    startangle=100, colors=sns.color_palette('Set2'))
plt.title('Rate of (not) transported passangers', fontsize=16)

display(passanger_transported.to_frame())
plt.show()

Видно, что данные разделены практически идеально поровну на две категории. Лишний раз стратифицировать данные, скорее всего, не придется при составлении train, val и test датасетов (хотя test сет уже есть). Да и в общем с классами равного размера проще работать.

---

## HomePlanet

In [ ]:
df_train['HomePlanet'].value_counts(dropna=False)

In [ ]:
# как работает crosstab
# (df_train.groupby('HomePlanet')['Transported']
#  .value_counts()
#  .sort_index(level=[0, 1])
#  .to_frame()
#  .rename(columns={'Transported': 'count'})
# )

In [ ]:
plot_size_multiple([df_train, df_test], 'HomePlanet', '', (0.01, 0.01, 0.01), [palette[0], palette[2], palette[1]])

In [ ]:
plot_tab('HomePlanet', True)

In [ ]:
df_survived_homeplanet = (df_train.groupby('HomePlanet')['Transported']
 .mean()
 .to_frame()
 .rename(columns={'Transported': 'transported_rate'})
 .sort_values(by=['transported_rate'])
)
df_survived_homeplanet

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
tab = pd.crosstab(df_train['HomePlanet'], df_train['Transported'])

bars = ax.bar(df_survived_homeplanet.index, df_survived_homeplanet.transported_rate, color=sns.color_palette('Set2'))
ax.grid(True, color='grey', axis='y', linestyle='-.', linewidth=0.5, alpha=0.6)
ax.set_xlabel('Home Planet', fontsize=12)
ax.set_ylabel('Rate of transported clients', fontsize=12)
ax.set_title('Rate of transported over home planets', fontsize=14)
ax.bar_label(bars, labels=[f'{y}/{x+y}' for x, y in tab[:].values], \
             label_type='center', fmt='%.2f', fontsize=14, color='white')

plt.show()

In [ ]:
df_train[df_train['HomePlanet'].isna()]['Transported'].mean()

In [ ]:
df_train.loc[df_train['HomePlanet'].isna(), 'HomePlanet'] = 'Mars'

In [ ]:
df_train['HomePlanet'].value_counts(dropna=False)

По графикам выше можно сделать несколько наблюдений:
* кол-во перелетов с Европы и с Марса довольно близко - $2131$ и $1759$ соответственно;
* больше всего путеществий наичанется с Земли - $4602$ перелета;
* чаще всего пассажиры успешно добирались до цели, стартуя с Европы - доля успехов около $2/3$;
* с Марса добираются до пунктов назначения примерно $50$% пассажиров (или скорее, что половина путешествий заканчивается успехом);
* чуть больше $40$% перелетов с Земли завершаются успешо.

Для всех имеющихся строк с пропуском в графе "HomePlanet" была посчитана средняя доля успешных перелетов, после чего все пропуски были добавлены к той их трех известных категорий, чей показатель среднего процента успешных перелетов был схожим. В данном случае это оказалась категория "Mars".

---

## CryoSleep

In [ ]:
df_train['CryoSleep'].value_counts(dropna=False)

In [ ]:
plot_size_multiple([df_train, df_test], 'CryoSleep', ['No CryoSleep', 'CryoSleep'], (0.01, 0), palette)

In [ ]:
# df_cryo_transp = (df_train.groupby('CryoSleep')['Transported']
#  .value_counts()
#  .sort_index(level=[0, 1])
#  .unstack(level=1)
# )
# df_cryo_transp

tab = pd.crosstab(df_train['CryoSleep'], df_train['Transported'])
tab

In [ ]:
plot_tab('CryoSleep')

In [ ]:
condition = df_train['CryoSleep'].isna()
selected_rows = df_train.loc[condition]

df_train.loc[condition, 'CryoSleep'] = \
    [True if tr else False for tr in selected_rows['Transported']]

In [ ]:
df_train['CryoSleep'].value_counts(dropna=False)

In [ ]:
df_train['CryoSleep'] = df_train['CryoSleep'].astype(bool)
df_train['CryoSleep'].dtype

Судя по графикам выше, можно предположить, что данный параметр является довольно важным для дальнейшей классификации полетов и определения таргета. Явно имеется некая закономерность, что перелеты, совершаемые в состоянии криосна, заканчиваются успехами намного чаще, чем перелеты не в криосне.

---

## Cabin

In [ ]:
df_train['Cabin'].isna().sum()

In [ ]:
df_train['Cabin'] = df_train['Cabin'].apply(lambda x: x[-1] if pd.notna(x) else x)
df_train['Cabin'].head()

In [ ]:
df_train['Cabin'].value_counts()

In [ ]:
plot_size('Cabin', ['Starboard', 'Port'], (0.01, 0), palette)

In [ ]:
plot_tab('Cabin')

In [ ]:
mask = df_train['Cabin'].isna()
nan_rows = df_train[mask]
split_point = len(nan_rows) // 2
nan_rows.loc[:, 'Cabin'] = ['P'] * split_point + ['S'] * (len(nan_rows) - split_point)

# df_train.update(nan_rows) # Почему-то когда я использую этот метод, теряются преобразования типов, сделанные ранее
df_train.loc[mask, 'Cabin'] = nan_rows
df_train['Cabin'].isna().sum()

Основная информация, несущая смысловую нагрузку данного параметра, потенциально полезная для будущей модели - это последняя буква в deck/num/side-коде кабины пассажира. Она говорит, с какой стороны расположена кабина (или место) пассажира. Решено оставить только эту букву, а всё до нее удалить.

Т.к. в датасете пассажиры равномерно распределены по двум возможным категориям (S и P), было решено разделить все пропуски поровну между этими двумя категориями.

---

## Destination

In [ ]:
df_train['Destination'].value_counts(dropna=False).to_frame()

In [ ]:
plot_size_multiple([df_train, df_test], 'Destination', '', (0.01, 0.01, 0.05), palette)

In [ ]:
df_train[df_train['Destination'].isna()]['Transported'].mean()

In [ ]:
plot_tab('Destination')

In [ ]:
df_survived_destination = (df_train.groupby('Destination')['Transported']
 .mean()
 .to_frame()
 .rename(columns={'Transported': 'transported_rate'})
 .sort_values(by=['transported_rate'])
)
df_survived_destination

Хоть средние показатели больше похожи на группу "PSO J318.5-22", но размеры группы не имеют порядковой разницы с количеством строк с пропуском. Поэтому присвоим всем пропускам значение самой крупной группы.

In [ ]:
df_train.loc[df_train['Destination'].isna(), 'Destination'] = 'TRAPPIST-1e'
df_train['Destination'].isna().sum()

Здесь рассуждения и наблюдения схожи с теми, что были проведены с параметром HomePlanet.

---

## Age

In [ ]:
df_train.Age.isna().sum()

In [ ]:
df_train.Age.describe().to_frame()

In [ ]:
with sns.axes_style("darkgrid"):
    fig, axs = plt.subplots(figsize=(8, 6), tight_layout=True)
    
    sns.histplot(data=df_train, x=df_train.Age, bins=20, kde=True, ax=axs)
    axs.set_title('Clients ages frequency', fontsize=16)
    axs.set_xlabel('Age', fontsize='14')
    axs.set_ylabel('Count', fontsize='14')
    axs.tick_params(axis='x', labelsize=11)
    axs.tick_params(axis='y', labelsize=11)

In [ ]:
plt.figure(figsize=(8, 6))
plt.boxplot(df_train.Age.dropna(), vert=False)
plt.title('Distribution of age among clients')
plt.xlabel('Age', fontsize=14)
plt.show()

In [ ]:
df_train.loc[df_train.Age.isna(), 'Age'] = df_train.Age.mean()
df_train.Age.isna().sum()

Средний возраст пассажиров - около 27-30 лет, при этом более молодых клиентов (до 30) больше, чем более возростных (старше 30), что видно на гистограмме выше. Имеется два пика - в районе 20-25 лет и, что интересно, в самом начале графика распределения - в районе 0-4 лет. Т.е. имеется значительное число пассажиров-детей. редкостью являются пассажиры старше 66 лет.

Заполняем пропуски в возрасте пассажиров средним значением, т.к. это проще всего, а количество пропусков мало.

---

## VIP

In [ ]:
df_train['VIP'].value_counts(dropna=False)

In [ ]:
df_train['VIP'].isna().sum()

In [ ]:
plot_tab('VIP', True)

In [ ]:
df_train['VIP'].fillna(value=False, inplace=True)

Во-первых, VIP-клиентов крайне мало - всего 199 человек на более чем 8000. Во-вторых, замечается некоторая закономерность, что VIP-клиенты на 10% менее вероятно успешно заканчивают перелет. Однако стоит учитывать малые размеры выборки данной категории клиентов, поэтому проверка наличия данной закономерности требует отдельных исследований и стат. экспериментов.

Пропуски было решено заполнить значением False, т.е. отсутствием VIP-статуса у пассажира. Данная категория пассажирова намного крупнее и будет менее подвержена стат. изменениям при добавлении новых значений, в отличае от группы VIP-клиентов. Кроме того, данный подход интуитивно логичнее.

---

## RoomService, FoodCourt, ShoppingMall, Spa, VRDeck

In [ ]:
columns = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

In [ ]:
df_train[columns].isna().sum().to_frame().rename(columns={0: 'nans'})

In [ ]:
df_train[columns].describe()

In [ ]:
df_train[df_train[columns] > 0][columns].describe()

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(16, 12), tight_layout=True)

i, j = 0, 0
for index, column in enumerate(columns):
    i, j = index // 2, index % 2
    rows = df_train[df_train[column] > 0][column]
    axs[i, j].boxplot(rows.dropna(), vert=False)
    axs[i, j].set_title(column)
    iqr = np.percentile(rows, 75) - np.percentile(rows, 25)
    axs[i, j].set_xlim(0, np.percentile(rows, 75) + 2 * iqr)
    
axs[i, j+1].set_visible(False)

In [ ]:
df_train.fillna({col: 0.0 for col in columns}, inplace=True)
df_train[columns].isna().sum().to_frame().rename(columns={0: 'nans'})

Платил - не платил, как выживаемость?

In [ ]:
all_inclusive_df = df_train.loc[(df_train[columns] > 0).all(axis=1)]
print(all_inclusive_df.shape[0], 'пассажира оплатило все возможные услуги.')

In [ ]:
all_inclusive_df[all_inclusive_df['VIP'] == True].shape[0]

In [ ]:
has_money_df = df_train.loc[(df_train[columns] > 0).any(axis=1)]
print(has_money_df.shape[0], 'пассажиров оплатило хотя бы одну из предоставляемых на борту услуг.')

In [ ]:
no_money_df = df_train.loc[(df_train[columns] == 0).all(axis=1)]
print(no_money_df.shape[0], 'пассажиров не оплатило ни одной из предоставляемых на борту услуг.')
print('Из них', no_money_df[no_money_df['CryoSleep'] == True].shape[0], 'пассажира находились в состоянии криосна.')

In [ ]:
print('Доля успешно закончивших путешествие пассажиров из числа оплативших каждую из доступных услуг:')
all_inclusive_df[all_inclusive_df['Transported'] == True].shape[0] / all_inclusive_df.shape[0]

In [ ]:
print('Доля успешно закончивших путешествие пассажиров из числа оплативших хотя бы одну из доступных услуг:')
has_money_df[has_money_df['Transported'] == True].shape[0] / has_money_df.shape[0]

In [ ]:
print('Доля успешно закончивших путешествие пассажиров из числа не оплативших ни одну из доступных услуг:')
no_money_df[no_money_df['Transported'] == True].shape[0] / no_money_df.shape[0]

In [ ]:
df_train['has_services'] = ~(df_train[columns] == 0).all(axis=1)
df_train.head()

Пропуски в любой из рассматриваемых пяти категорий заполнены нулями по тому принципу, что раз уж никакой информации нет, то логичнее предположить, что заполнять было нечего, и вместо нуля оставили пустое значение.

Пассажиры были разделены по степени заинтересованности в услугах, а скорее по степени трат на эти услуги. Выделено три категории: пассажиры, оплатившие хотя бы одну услугу, все услуги или ни одной. Пассажиров из последней категории оказалось больше всего - порядка 57% (5040/8693). Пассажиров из первой категории тоже оказалось достаточно много - 42% от всей выборки (3653/8693). Клиентов, оплативших каждую из услуг - всего 252 человека или около 6% от размеров предыдущей категории.

После такого разделения была изучен процент успешно завершенных путешествий внутри каждой из категорий. Было замечено, что, как ни странно, пассажиры, имеющие оплаченные услуги, реже заканчивают путешествие - доля успехов составляет около 30%. В то же время пассажиры без оплаченных услуг в почти 80% случаев завершают перелет без проблем. Однако, было также замечено, что более 80% пассажиров без услуг во время перелета находились в состоянии криосна, а как было замечено в соответствующем пункте анализа, имеется некоторя корреляция между этим показателем и успешностью перелета. Поэтому можно предположить, что признак наличия оплаченных услуг является косвенным признаком успешности путешествия. 

А раз важно само наличие оплаченных услуг, решено упростить 5 категорий в одну - имелись ли хоть какие-то оплаченные услуги у пассажира. Именно в такой формулировке принято обобщить рассматриваемые категории.

---

## Name

In [ ]:
df_train.Name.isna().sum()

In [ ]:
mask = df_train['Name'].isna()
l = df_train[mask].shape[0]
df_train.loc[mask, 'Name'] = ['noname_' + str(i) for i in range(0, l)]
df_train.Name.isna().sum()

In [ ]:
df_train['Name'].duplicated(keep=False).sum()

In [ ]:
df_train[df_train['Name'].duplicated(keep=False)].sort_values(by='Name')

In [ ]:
df_train.drop_duplicates(subset=['Name'], keep='last', inplace=True)
df_train['Name'].duplicated().sum()

Имя - почти как уникальный идентификатор, логично предположить, что особой ценности для предсказания целевого параметра оно не имеет. Поэтому в дальнейшем от данного параметра мы избавимся, а сейчас удалим все строчки, в которых повторяется имя одного и того же пассажира, оставив последнюю запись. Последнюю в предположении, что у записей имеется какой-нибудь хронологический порядок, тогда последняя запись о пассажире - самая актуальная. Строк с дубликатами имен оказалось незначительно мало, поэтому даже если идея неверна, она вряд ли повлияет на конечное качество данных.

Пропуски имен были заполнены значениями по умолчанию.

---

# Оставшиеся пропуски

In [ ]:
df_train.isna().sum()
df_test.isna().sum()

---

## Correlation matrix

In [ ]:
df_train.reset_index(drop=True, inplace=True)
df_train.drop(columns=['PassengerId', 'Name'], inplace=True)
df_train['Transported'] = df_train['Transported'].astype(object)
df_train['VIP'] = df_train['VIP'].astype(object)
df_train['CryoSleep'] = df_train['CryoSleep'].astype(object)
df_train['has_services'] = df_train['has_services'].astype(object)
df_train.info()

In [ ]:
interval_cols = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

correlation_matrix_train = df_train.phik_matrix(interval_cols=interval_cols)

In [ ]:
plt.figure(figsize=(10, 8))

sns.heatmap(correlation_matrix_train, annot=True, cmap='coolwarm', fmt='.2f', cbar_kws={"shrink": 0.75})

plt.title('Phi-K корреляционная матрица')
plt.tick_params(axis='x', labelsize=12)
plt.tick_params(axis='y', labelsize=12)
plt.show()

Корреляционная матрица показывает наличие слабой корреляции между параметром "CryoSleep" и целевым значением, как и предполагалось ранее. Также видна слабая обратная корреляция между параметров "has_services" и целевым значением, что тоже было ожидаемо, т.к. "CryoSleep" и "has_services" в свою очередь имеют сильную обратную корреляцию. Остальные параметры с целевым признаком имеют незначительную, почти незаметную связь.

В конце анализа данных и их подготовки решено удалить столбцы с ID и именами пассажиров и все колонки с услугами, т.к. мы их упростли до одного значения. Все категориальные параметры были приведены к числовым значениям с помощью label-encoding и one-hot-encoding. Label-encoding применен на колонках HomePlanet, Cabin, Destination, has_services и VIP, т.к. эти параметры имеют слабую связь с целевым параметром. One-hot-encoding был использован на колонке CryoSleep, т.к. данный параметр имеет заметную корреляцию с целевым параметром. Значения параметра Age не изменены. (Наверное можно избавиться и от параметра has_services, т.к. он коррелирует с CryoSleep, а это, как я помню, приводит к проблеме multicollinearity.)

Итого остается 8 параметров, из которых показательными могут быть has_services и CryoSleep. Остальные колонки оставлены по остаточному принципу, т.к. вкупе могут составить связь, коррелирующую с целевым параметром. 

---

# Test set

In [ ]:
df_test[df_test.columns].isna().sum().to_frame().rename(columns={0: 'count'})

In [ ]:
df_test.drop(columns=['Name', 'PassengerId'], inplace=True)
df_test.columns

In [ ]:
df_test['has_services'] = (df_test[columns] > 0).any(axis=1)

important_columns = ['HomePlanet', 'Cabin', 'Destination', 'Age', 'VIP', 'has_services', 'CryoSleep']
df_test[important_columns].isna().any(axis=1).sum()

In [ ]:
# df_test.drop(columns=columns, inplace=True)
df_test.drop(df_test[df_test[important_columns].isna().any(axis=1)].index, inplace=True)
df_test.head()

In [ ]:
df_test['Cabin'] = df_test['Cabin'].apply(lambda x: x[-1] if pd.notna(x) else x)

In [ ]:
print('Total number of rows:', df_test.shape[0])
print('Number of rows without any gaps:', df_test[df_test.columns].notna().all(axis=1).sum())
print('Number of rows with gap in any column:', df_test[df_test.columns].isna().any(axis=1).sum())
print('Percentage of rows without any gaps:', df_test[df_test.columns].notna().all(axis=1).mean())

In [ ]:
df_test.drop(df_test[df_test[df_test.columns].isna().any(axis=1)].index, inplace=True)
df_test.shape[0]

In [ ]:
display(df_train.dtypes.to_frame(), df_test.dtypes.to_frame())

Над test датасетом были проведены аналогичные с train манипуляции, дабы привести их к одному виду.

Т.к. train set является валидационным инструментом нашей будущей модели, изменять значения в нем или добавлять их было бы скорее неправильно, т.к. тогда мы бы могли испортить реальную картину, которую показывают эти данные. Т.к. пропуски в train составляют чуть более 10% от более чем 4000 записей, удаление всех строк с пропуском в хотя бы одной из интересующих нас колонкок не приведет к значительным потерям.

---

# Сохранение очищенных данных

In [ ]:
path1 = 'datasets/train_cleaned.csv'
path2 = 'datasets/test_cleaned.csv'

df_train.to_csv(path1, index=False)
df_test.to_csv(path2, index=False)